**ReadMe**

The part 1 is a discussion on the use of tabular algorithm.

The question 1 is answered in part 2.

The question 2 is answered in part 2.

The question 3 is answered in part 2.

In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
%cd /content/drive/My\ Drive/Winter\ 2022/CME241

Mounted at /content/drive
/content/drive/My Drive/Winter 2022/CME241


In [21]:
from rl.distribution import *
from rl.function_approx import *
from rl.markov_decision_process import *
from rl.policy import *
from rl.dynamic_programming import *
from rl.monte_carlo import *
from rl.chapter2.simple_inventory_mrp import *
import numpy as np
from itertools import islice
from rl.gen_utils.plot_funcs import plot_list_of_curves
import matplotlib.pyplot as plt
import time
from typing import Tuple, Sequence, Iterator, List
from __future__ import annotations

#1. Why using Tabular \& Limitations? 

In the second part of this assignment, we will deal with the case of finite Markov reward process. Although the fact that the process is finite does not necessarily force us to use tabular algorithm in approximating the dynamics, it is one of the most straightforward ways to understand the expected reward while also not implicitly enforcing any parametric guess on the form of the predicted value.[link text](https://)

The tabularness of the prediction will allow us to treat each data input (which has finite variations) as an almost disconnected value in the sense that, even if this data input is similar to another in the input space, we will not enforce any guess that it will also look similar. Although this assumption of non-assuming similarity for similar input may sounds bad for our prediction since we expect similar things to yield similar behavior, this may not be true especially when we deal with a system with high sensitivity in which small change in the input can result in a tremendous change in the output. Moreover, assuming dependence upon neighbor can also introduce bias into the model, as what we predict for this case will rather also use the observation from other cases.

Another use case for the tabular algorithm is when we do not know about the input. For example, we may not know how to measure the distance or the similarity of the inputs.

For example, if we want to estimate the willing to pay prices for our products, which are cars, chopsticks, and sneakers. It is possible likely that sneakers maybe more similar to cars than to chopsticks but it is also quite possible in the differ\ence case. The similarity in this sense may be hard to qunatify and hard to be used. Therefore, even though we have several datapoints for bith cars and chopsticks but a very few datapoints for sneakers, we may not want to interpolate the relationship from the cars and chopsticks into the sneakers.

However, this market model has been heavily challenged by the recent growth of online markets.

We can see that in online marketplace, although the total number of items are still finite, the number can be very large. Moreover, the purchase can be sparse. (When a person tries to buy some items from Amazon, he may search on the product he wants to look into and gets to see hundreds of products. He eventually choose to purchase only one of them or maybe non of them.)  Therefore, if we treat each item as a discrete point without any interaction, we may get a very unreliable model. Moreover, as the end objective of the consumer demand estimation is not to actually know the demand, but to rather use the demand to understand what will happen if we launch a new product of this characteristics or what will ahppen if we put some promotion upon this products. We want to get a counterfactual prediction, in which the product we ask the model to predict for us has never been seen by the model before. 

(We say that imposing a promotion can be treated as a new product as the price will change so the product can be said to be difference. In some model consideration, people will only discretize the item input by only the phsyical characteristics but not the price.)

One technique that we can do is to just think about what is the most similar product to this one and use the same value as a prediction. Another naive way is to find k most similar product and take an average or a weughted average. This is indeed the same as KNN (k-Nearest Neighbor). In KNN such as in image classifier, the input is an image that we can quantify as a vector of pixel values. However, in the training process, we do not use those value at all, and we do proceed to evaluate each item individually. Since we discretize directly by putting image with exacty the same vector representation into 1 point (in the table), we will see that one table will only see 1 instance! (since it is quite hgard for two images to have the same vector representations in the highdimensional vector space). Thus, the training is to observe that 1 value per 1 input in the table, the best way to do so is then to take taht value as the predicted value. Therefore, the trainign of the KNN does not require any actual computation, but only requires the storing of the big table of the same size as teh data input itself.

In the evaluation stage, we input an image to the KNN. It is likelythat the image we ask the model to predict will not have any overlapping with the training data point, so we fidn the k-nearest neghbor and find the average prediction instead.

For example, let consider an easier to understand case.

If we try to predict a curvature of a two dimensional curve at different x value.

If we want to assume some regulation on the conitnuity of the curve, we will see that if we perturb the value of x to be $x+\delta x$, we will expect to see that the crosssections at the two x values will be only alittle bit different. Therefore, we can use the data of observation at x to help the prediction at $x+\delta x$ even though it is not at the exactlyt same spot. For example, one can do so by adding regularization while training.

Note that in the neural networks model where we impose the L2 regularization can also be thought of that we want similar input to not have so stark difference. For example if the model is
\begin{align*}
  y = wx
\end{align*}
Therefore,
\begin{align*}
  \vert \Delta y \vert  = \vert w \vert \vert \Delta x \vert 
\end{align*}

Thus, as the regularization tries to push the value of $w$ to be closer top zero. The relative difference will also be smaller.

This, therefore, will help reducing variance in the prediction. However, if we want to model a system where we have already know that there should be a great variation (such as modelling rectangular wave), the regularization can lead to serious bias.

For physics example, we can see that the nature has a regularity enforcement upon most mechanical system. For example, the speed cannot be faster than the speed of light. Therefore, we will know for sure that the displacement of an object of interests cannot be more than $ct$.

Therefore, if we observe the location of a car at time = 8.00am, we will be able to bound the possible location of the car at time = 1.00pm. 

The question is that will we use that bound to guide our prediction. 

The answer is clearly "No."

Within that time span of three hours, the car (if only limited by the speed of lights, can go back and forth from the earth to the sun for more than ten times! Therefore, this bound will says that any place in the world is possible.

#2. Simple Inventory Self-Dynamics via Reinforcement Learning

We, first, denote that this problem will be a case when we do not impose any intervention into the system. In other words, we will not submit any more order for the store. Therefore, we will be able to observe the self-dynamics of the inventory.

We first initialize the simple inventory model.

In [6]:
user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0
user_gamma = 0.9

si_mrp = SimpleInventoryMRPFinite(
        capacity=user_capacity,
        poisson_lambda=user_poisson_lambda,
        holding_cost=user_holding_cost,
        stockout_cost=user_stockout_cost
    )

We first have a look at its transitions, we can see that the non-terminal nodes that cannot be achieve by starting at any other places is when we have 2 onhand and 0 in store. We therefore choose to use this point as a start point of the reward traces generators. However, we can use randomization fro the reward traces if we want.

In [17]:
si_mrp

From State InventoryState(on_hand=0, on_order=0):
  To [State InventoryState(on_hand=0, on_order=2) and Reward -10.000] with Probability 1.000
From State InventoryState(on_hand=0, on_order=1):
  To [State InventoryState(on_hand=1, on_order=1) and Reward -0.000] with Probability 0.368
  To [State InventoryState(on_hand=0, on_order=1) and Reward -3.679] with Probability 0.632
From State InventoryState(on_hand=0, on_order=2):
  To [State InventoryState(on_hand=2, on_order=0) and Reward -0.000] with Probability 0.368
  To [State InventoryState(on_hand=1, on_order=0) and Reward -0.000] with Probability 0.368
  To [State InventoryState(on_hand=0, on_order=0) and Reward -1.036] with Probability 0.264
From State InventoryState(on_hand=1, on_order=0):
  To [State InventoryState(on_hand=1, on_order=1) and Reward -1.000] with Probability 0.368
  To [State InventoryState(on_hand=0, on_order=1) and Reward -4.679] with Probability 0.632
From State InventoryState(on_hand=1, on_order=1):
  To [State I

In [18]:
si_mrp.non_terminal_states

[NonTerminal(state=InventoryState(on_hand=0, on_order=0)),
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)),
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)),
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)),
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)),
 NonTerminal(state=InventoryState(on_hand=2, on_order=0))]

In [19]:
reward_traces_gen = si_mrp.reward_traces(si_mrp.non_terminal_states[2])

In [14]:
si_mrp.non_terminal_states[2]

NonTerminal(state=InventoryState(on_hand=0, on_order=2))

In [25]:
mc_func_approx = LinearFunctionApprox.create(feature_functions=[(lambda x, s=s: float(x.state == s.state)) for s in si_mrp.non_terminal_states],
        adam_gradient=AdamGradient(
    learning_rate=0.05,
    decay1=0.9,
    decay2=0.999
)
    )

In [36]:
what = mc_prediction(reward_traces_gen, mc_func_approx, 0.3)

In [39]:
episodes: Iterator[Iterator[mp.ReturnStep[S]]] = \
        (returns(trace, 0.3, 1e-6) for trace in reward_traces_gen)

In [41]:
next(episodes)

AttributeError: ignored

In [38]:
next(what)

AttributeError: ignored

In [ ]:
def mc_prediction(
    traces: Iterable[Iterable[mp.TransitionStep[S]]],
    approx_0: ValueFunctionApprox[S],
    γ: float,
    episode_length_tolerance: float = 1e-6
) -> Iterator[ValueFunctionApprox[S]]:
    '''Evaluate an MRP using the monte carlo method, simulating episodes
    of the given number of steps.

    Each value this function yields represents the approximated value
    function for the MRP after one additional epsiode.

    Arguments:
      traces -- an iterator of simulation traces from an MRP
      approx_0 -- initial approximation of value function
      γ -- discount rate (0 < γ ≤ 1), default: 1
      episode_length_tolerance -- stop iterating once γᵏ ≤ tolerance

    Returns an iterator with updates to the approximated value
    function after each episode.

    '''
    episodes: Iterator[Iterator[mp.ReturnStep[S]]] = \
        (returns(trace, γ, episode_length_tolerance) for trace in traces)
    f = approx_0
    yield f

    for episode in episodes:
        f = last(f.iterate_updates(
            [(step.state, step.return_)] for step in episode
        ))
        yield f


#1. Univariate BSpline